In [2]:
import pandas as pd

dataset = pd.read_csv('data/MovieLens/dates.csv')
movies = pd.read_csv('data/MovieLens/movies.csv')
ratings1 = pd.read_csv('data/MovieLens/ratings1.csv')
ratings2 = pd.read_csv('data/MovieLens/ratings2.csv')

In [3]:
movies['title'].nunique()
ratings1['userId'].nunique()

274

In [4]:
dataset['date'] = pd.to_datetime(dataset['date'])
dataset['year'] = dataset['date'].dt.year
dataset['year'].mode()

0    2000
Name: year, dtype: int32

In [5]:
ratings = pd.concat([ratings1, ratings2], ignore_index=True)
display(ratings)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100832,610,166534,4.0
100833,610,168248,5.0
100834,610,168250,5.0
100835,610,168252,5.0


In [6]:
ratings = ratings.drop_duplicates(ignore_index=True)
print('Число строк в таблице ratings: ', ratings.shape[0])
# Число строк в таблице ratings: 100836

Число строк в таблице ratings:  100836


In [7]:
ratings_dates = pd.concat([ratings, dataset], axis=1)
display(ratings_dates.tail(7))

,userId,movieId,rating,date,year
100829,610,164179,5.0,2017-05-03 21:07:11,2017
100830,610,166528,4.0,2017-05-04 06:29:25,2017
100831,610,166534,4.0,2017-05-03 21:53:22,2017
100832,610,168248,5.0,2017-05-03 22:21:31,2017
100833,610,168250,5.0,2017-05-08 19:50:47,2017
100834,610,168252,5.0,2017-05-03 21:19:12,2017
100835,610,170875,3.0,2017-05-03 21:20:15,2017


In [8]:
import pandas as pd

items_df = pd.DataFrame({
            'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394],
            'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
            'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
        })

purchase_df = pd.DataFrame({
            'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
            'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132],
            'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
        })


# 1. Объединение таблиц по item_id
merged = items_df.merge(purchase_df, on='item_id', how='inner')
display(merged)
temp = merged.copy()

temp['total_price'] = merged['price'] * merged['stock_count']

# 2. Расчёт общего дохода (только по совпавшим item_id)
income = temp['total_price'].sum()

print(income)


,item_id,vendor,stock_count,purchase_id,price
0,417283,Samsung,54,101,13900
1,849734,LG,33,101,5330
2,132223,Apple,122,101,38200
3,573943,Apple,18,112,49990
4,19475,LG,102,121,9890
5,3294095,Apple,43,145,33000
6,382043,Samsung,77,145,67500
7,302948,Samsung,143,145,34500
8,100132,LG,60,221,11400


19729490


# new file csv merged

In [9]:
total_movies = pd.read_csv('data/MovieLens/ratings_movies.csv')
total_movies.head()

,Unnamed: 0,userId,movieId,rating,date,title,genres
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [10]:
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

Задание 8.1
1 point possible (graded)
Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
У скольких фильмов не указан год их выпуска?

In [11]:
total_movies['year_release'] = total_movies['title'].apply(get_year_release)
total_movies.describe()

print(total_movies['year_release'].isna().value_counts())

year_release
False    100818
True         18
Name: count, dtype: int64


Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
В качестве ответа запишите название этого фильма без указания года его выпуска.

In [ ]:
# 1. Фильтруем фильмы за 1999 год
movies_1999 = total_movies[total_movies['year_release'] == 1999]

# 2. Строим pivot_table
pivot_1999 = movies_1999.pivot_table(
    values='rating',
    index='title',
    aggfunc='median'
)

# 3. Сортируем по рейтингу (по возрастанию или укажи ascending=False для убывания)
pivot_1999_sorted = pivot_1999.sort_values(by='rating')

# 4. Вывод
print(pivot_1999_sorted)



                                          rating
title                                           
Bloodsport: The Dark Kumite (1999)           0.5
Trippin' (1999)                              1.0
Source, The (1999)                           1.0
Chill Factor (1999)                          1.0
Simon Sez (1999)                             1.0
...                                          ...
Larry David: Curb Your Enthusiasm (1999)     5.0
On the Ropes (1999)                          5.0
Mickey's Once Upon a Christmas (1999)        5.0
Trailer Park Boys (1999)                     5.0
Sun Alley (Sonnenallee) (1999)               5.0

[261 rows x 1 columns]


Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

In [19]:
movies_2010 = total_movies[total_movies['year_release'] == 2010]

pivot_2010 = movies_2010.pivot_table(
    values='rating',
    columns='genres',
    aggfunc='median'
)

# 3. Сортируем по рейтингу (по возрастанию или укажи ascending=False для убывания)
pivot_2010_sorted = pd.melt(pivot_2010).sort_values(by='value')

pivot_2010_sorted



,genres,value
46,Action|Sci-Fi,1.00
14,Action|Adventure|Horror,1.50
36,Action|Drama|Fantasy,1.50
87,Crime|Romance,1.50
55,Adventure|Comedy|Fantasy,1.75
...,...,...
82,Crime,4.75
52,Adventure|Children|Comedy|Mystery,5.00
65,Animation|Children|Mystery,5.00
67,Animation|Drama|Fantasy|Mystery,5.00


In [20]:
total_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    100836 non-null  int64  
 1   userId        100836 non-null  int64  
 2   movieId       100836 non-null  int64  
 3   rating        100836 non-null  float64
 4   date          100836 non-null  object 
 5   title         100836 non-null  object 
 6   genres        100836 non-null  object 
 7   year_release  100818 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.2+ MB


In [21]:
# Предположим, df — твой DataFrame
# 1. Создаём сводную таблицу: строки — userId, колонки — genres, значения — количество просмотров
pivot = total_movies.pivot_table(
    index='userId',
    columns='genres',
    values='movieId',
    aggfunc='count',
    fill_value=0
)

# 2. Превращаем значения в 1/0 — смотрел или нет
pivot_binary = pivot.applymap(lambda x: 1 if x > 0 else 0) # pyright: ignore[reportCallIssue]

# 3. Считаем, сколько уникальных жанровых комбинаций смотрел каждый пользователь
genre_combinations_count = pivot_binary.sum(axis=1)

# 4. Находим userId с наибольшим числом уникальных комбинаций жанров
top_user = genre_combinations_count.idxmax()

print("🔎 Пользователь с наибольшим числом уникальных комбинаций жанров:", top_user)


C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_18004\2433144061.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_binary = pivot.applymap(lambda x: 1 if x > 0 else 0) # pyright: ignore[reportCallIssue]


🔎 Пользователь с наибольшим числом уникальных комбинаций жанров: 599


In [22]:
pivot = total_movies.pivot_table(
    index='userId',
    values='rating',
    aggfunc={'rating': ['count', 'mean']}
)

# Убираем многоуровневость и задаём читаемые имена колонкам
pivot.columns = ['rating_count', 'rating_mean']

# 1. Находим минимальное количество оценок
min_count = pivot['rating_count'].min()

# 2. Фильтруем пользователей с этим числом оценок
min_count_users = pivot[pivot['rating_count'] == min_count]

# 3. Находим пользователя с максимальной средней оценкой среди них
top_user = min_count_users['rating_mean'].idxmax()

print("🔎 Пользователь:", top_user)


🔎 Пользователь: 53


In [26]:
# Фильтрация по 2018 году
df_2018 = total_movies[total_movies['year_release'] == 2018]

# Сводная таблица: genres -> count и mean по рейтингу
pivot = df_2018.pivot_table(
    index='genres',
    values='rating',
    aggfunc={'rating': ['count', 'mean']}
)

# Упрощаем названия колонок
pivot.columns = ['rating_count', 'rating_mean']

# Оставляем только те жанры, у которых больше 10 оценок
filtered = pivot[pivot['rating_count'] > 10]

# Сортировка по убыванию средней оценки
sorted_pivot = filtered.sort_values(by='rating_mean', ascending=False)

# Выводим таблицу
print(sorted_pivot.head(10))  # или без .head() — покажет всю таблицу


                         rating_count  rating_mean
genres                                            
Action|Adventure|Sci-Fi            14     3.928571
Action|Comedy|Sci-Fi               12     3.875000
